# Online Retail Project

# Preparation

In [1]:
# Install and import required libraries
# pyfpgrowth(https://fp-growth.readthedocs.io/en/latest/)
!pip install pyfpgrowth # ! is needed for Google Colab, it is optional for GCP
import os
import pyfpgrowth
import pandas as pd

     |████████████████████████████████| 1.6 MB 4.1 MB/s 
  Created wheel for pyfpgrowth: filename=pyfpgrowth-1.0-py2.py3-none-any.whl size=5504 sha256=3d4f2c91372acf1bd7b59c65c6a5a089709e90ca2803c7e9f5a87f8480d712ef
  Stored in directory: /root/.cache/pip/wheels/73/97/4b/f12ac994f6bbb99597396255435824c73ad3916be1e678be55
Successfully built pyfpgrowth


# Dataset Description
[Online Retail](http://archive.ics.uci.edu/ml/datasets/online+retail#) contains all the transactions occurring between 01-12-2010 and 09-12-2011 for a UK-based and registered non-store online retail. The purpose of this project is to extract Association Rules by using Frequent Patterm Mining algorithm (FPGrowth Tree).

## Dictionary
- InvoiceNo: Invoice number. Nominal, a 6-digit integral number uniquely assigned to each transaction. If this code starts with letter 'c', it indicates a cancellation. 
- StockCode: Product (item) code. Nominal, a 5-digit integral number uniquely assigned to each distinct product. 
- Description: Product (item) name. Nominal. 
- Quantity: The quantities of each product (item) per transaction. Numeric.	
- InvoiceDate: Invice Date and time. Numeric, the day and time when each transaction was generated. 
- UnitPrice: Unit price. Numeric, Product price per unit in sterling. 
- CustomerID: Customer number. Nominal, a 5-digit integral number uniquely assigned to each customer. 
- Country: Country name. Nominal, the name of the country where each customer resides.

## 1
Download the Online Retail data from given link and read it as Pandas DataFrame. Display the first few rows of the data. Examine its properties such as rows, columns, any missing values and columns names. Read the data dictionary and confirm your understanding of each column.

In [2]:
# Following shell commands will download the data and list contents of directory. 
!wget -q -N 'http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx'
!ls 
# For convenience the file name is changed.
!mv 'Online Retail.xlsx' OnlineRetail.xlsx

'Online Retail.xlsx'   sample_data


In [3]:
# read_excel function reads excel sheet as a DataFrame
dataset = pd.read_excel('OnlineRetail.xlsx')

In [4]:
dataset.head(5)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [5]:
# check shape (rows, columns) of dataset
print(dataset.shape)
# check null (missing) values in individual columns
print(dataset.isnull().sum())
# check null (missing) values in the dataset
print(dataset.isnull().sum().sum()) # 136534
# The columns names
print(dataset.columns)

(541909, 8)
InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64
136534
Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')


## 2
InvoiceNumber column provides a unique id to each transaction. Some invoice number start with 'c' indicating cancelled transactions. Filter cancelled transactions from your dataset. Then, generate frequency count of items in each transaction. Create a list of unique Invoices that have more than 5 and less than 10 items. Final data must not include cancelled transactions and transactions with less than 5 or more than 10 items. 

In [6]:
# If InvoiceNo starts with letter 'c', it indicates a cancellation.
dataset_x = dataset[~ dataset['InvoiceNo'].str.startswith('C', na=False)] # filter 9288 cancelled or NaN transations
# Count cancelled invoices
dataset_x['InvoiceNo'].count() # 532621 left out of 541909

532621

In [7]:
# filter transaction where items baught are less than 5
min_items_in_transaction = 5
max_items_in_transaction = 10
InvoiceNo_Count = dataset_x['InvoiceNo'].value_counts() # 25900 transaction
InvoiceNo_List = list(set(InvoiceNo_Count[InvoiceNo_Count.between(min_items_in_transaction, max_items_in_transaction)].index.tolist()))
dataset_x2 = dataset_x[dataset_x['InvoiceNo'].isin(InvoiceNo_List)]
print('transactions in final dataset', dataset_x2.shape[0])
print('Invoices', len(InvoiceNo_List))

transactions in final dataset 26975
Invoices 3646


## 3
Transform your filtered dataset into two python dictionaries. Each item of the first dictionary StockCode2Alias_Dict will have StockCode as key and a unique-id as value. Each item of the second dictionary Alias2StockCode_Dict will have unique-id as key and StockCode and Description as value. A StockCode can have more than one descriptions. Keep the first description of each StockCode.

In [8]:
# print(dataset_x2[dataset_x2['StockCode'] == '85123A']['Description'].unique())
StockCode2Alias_Dict = {}
Alias2StockCode_Dict = {}
StockCodes = list(dataset_x2['StockCode'].unique())
for c in StockCodes:
  if c not in StockCode2Alias_Dict.keys():
    # add item to StockCode2Alias_Dict
    StockCode2Alias_Dict[c] = len(StockCode2Alias_Dict) # StockCode_Num_Alias
    # keep the first descriotion
    desc = dataset_x2[dataset_x2['StockCode'] == c]['Description'].unique()[0]
    # add item to Alias2StockCode_Dict
    Alias2StockCode_Dict[StockCode2Alias_Dict[c]] = (c, desc) # StockCode_Num_Alias
# print(StockCode2Alias_Dict)
print(Alias2StockCode_Dict)

{0: ('85123A', 'WHITE HANGING HEART T-LIGHT HOLDER'), 1: (71053, 'WHITE METAL LANTERN'), 2: ('84406B', 'CREAM CUPID HEARTS COAT HANGER'), 3: ('84029G', 'KNITTED UNION FLAG HOT WATER BOTTLE'), 4: ('84029E', 'RED WOOLLY HOTTIE WHITE HEART.'), 5: (22752, 'SET 7 BABUSHKA NESTING BOXES'), 6: (21730, 'GLASS STAR FROSTED T-LIGHT HOLDER'), 7: (22783, 'SET 3 WICKER OVAL BASKETS W LIDS'), 8: (22961, 'JAM MAKING SET PRINTED'), 9: (22960, 'JAM MAKING SET WITH JARS'), 10: (22663, 'JUMBO BAG DOLLY GIRL DESIGN'), 11: ('85049A', 'TRADITIONAL CHRISTMAS RIBBONS'), 12: (22168, 'ORGANISER WOOD ANTIQUE WHITE '), 13: (22662, 'LUNCH BAG DOLLY GIRL DESIGN'), 14: (79321, 'CHILLI LIGHTS'), 15: (22780, 'LIGHT GARLAND BUTTERFILES PINK'), 16: (22779, 'WOODEN OWLS LIGHT GARLAND '), 17: (22466, 'FAIRY TALE COTTAGE NIGHTLIGHT'), 18: (21731, 'RED TOADSTOOL LED NIGHT LIGHT'), 19: (22150, '3 STRIPEY MICE FELTCRAFT'), 20: (22619, 'SET OF 6 SOLDIER SKITTLES'), 21: (21891, 'TRADITIONAL WOODEN SKIPPING ROPE'), 22: (21889, '

## 4
Create a list and name it as transactions. Append the items from InvoiceNo_List, created in step 2, to it. Each item in transactions contains list of StockCodes of the items bought.

In [9]:
transactions = []
for invoice_no in InvoiceNo_List:
    items = dataset_x2[dataset_x2['InvoiceNo'] == invoice_no]['StockCode'].to_list()   
    transactions.append([StockCode2Alias_Dict[i] for i in items])    
print('transactions_db size', len(transactions))
print(transactions[0:9])

transactions_db size 3646
[[2074, 2073, 1282, 380, 286, 215, 452, 1829], [2562, 2521, 18, 35, 38, 124, 1861], [512, 1114, 1570, 1568, 1842], [1323, 1001, 1136, 1629, 1630, 1631, 1631], [1577, 198, 197, 768, 20, 21, 799], [1836, 1990, 1981, 1992, 1989, 577, 2019, 1391], [1007, 736, 992, 1008, 1009, 1010, 729, 962], [146, 1235, 1314, 126, 137, 583, 9, 1959], [215, 158, 195, 822, 419, 46, 317]]


## 5
Extract Association Rules by using Frequent Patterm Mining algorithm (FPGrowth Tree). The trasactions list is going to be the input to the algorithm. Print the extracted rules with the help of Alias2StockCode_Dict created in step 3.

In [10]:
patterns = pyfpgrowth.find_frequent_patterns(transactions, 5) # try different support threshold values

In [11]:
rules = pyfpgrowth.generate_association_rules(patterns, 1) # try different probability threshold values
# rules are a dictionary of antecedents and consequents.

In [12]:
for antecedents, consequents in rules.items():
    antec_list =[]
    conseq_list =[]
    for a in antecedents:
        antec_list.append(Alias2StockCode_Dict[a][1])
    for c in consequents[0]:
        conseq_list.append(Alias2StockCode_Dict[c][1])
    print(antec_list, '-->', conseq_list, consequents[1])  

['SMALL CHOCOLATES PINK BOWL'] --> ['SMALL DOLLY MIX DESIGN ORANGE BOWL'] 1.0
['HANGING HEART BONHEUR'] --> ['HOME SWEET HOME HANGING HEART'] 1.0
['HERB MARKER ROSEMARY', 'HERB MARKER PARSLEY'] --> ['HERB MARKER BASIL'] 1.0
['HERB MARKER BASIL', 'HERB MARKER PARSLEY'] --> ['HERB MARKER THYME'] 1.0
['HERB MARKER THYME', 'HERB MARKER PARSLEY'] --> ['HERB MARKER BASIL'] 1.0
["POPPY'S PLAYHOUSE BATHROOM"] --> ["POPPY'S PLAYHOUSE BEDROOM ", "POPPY'S PLAYHOUSE KITCHEN"] 1.0
["POPPY'S PLAYHOUSE LIVINGROOM ", "POPPY'S PLAYHOUSE BATHROOM"] --> ["POPPY'S PLAYHOUSE BEDROOM ", "POPPY'S PLAYHOUSE KITCHEN"] 1.0
["POPPY'S PLAYHOUSE BEDROOM ", "POPPY'S PLAYHOUSE BATHROOM"] --> ["POPPY'S PLAYHOUSE KITCHEN"] 1.0
["POPPY'S PLAYHOUSE KITCHEN", "POPPY'S PLAYHOUSE BATHROOM"] --> ["POPPY'S PLAYHOUSE BEDROOM "] 1.0
["POPPY'S PLAYHOUSE BEDROOM ", "POPPY'S PLAYHOUSE LIVINGROOM ", "POPPY'S PLAYHOUSE BATHROOM"] --> ["POPPY'S PLAYHOUSE KITCHEN"] 1.0
["POPPY'S PLAYHOUSE LIVINGROOM ", "POPPY'S PLAYHOUSE KITCHEN", "P

This is the end of Lab 3

**Ceni Babaoglu, PhD**

The Chang School of Continuing Education

Ryerson University